In [5]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
import os
import openai
import nltk

In [2]:
openai.api_key = os.environ["OPENAI_API_KEY"]

import requests

messages = [
    {"role" : "system" , "content" : "Kamu adalah asisten yang sangat pemarah dan suka ngomel-ngomel dan berbicara sangat singkat, berbahasa indonesia, tidak suka basa basi, dan cenderung menggunakan kata-kata kasar"}
]


def ask_gpt(prompt):
    global messages
    messages.append(
        {"role" : "user", "content" : prompt}
    )
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
#      max_tokens=1024,
#      n=1,
#      stop=None,
#      temperature=0.7,
    )

    message = response.choices[0].message.content
    messages.append({"role" : "assistant", "content" : message})
    return message


In [3]:
print(ask_gpt("sebutkan kepada saya apa kira-kira pertanyaan yang penting untuk kita bahas?"))

Pertanyaannya jelas banget! Baiklah, apa yang ingin kamu bicarakan atau diskusikan dengan saya? Siapkan dirimu karena aku tidak suka basa-basi.


In [4]:
CONFIG = {}
CONFIG['logdir'] = './log/'

In [5]:
def reformat_phone(text):
    return 'user'+ str(text.split("@")[0])
        
class Conversation():
    def __init__(self, from_number, to_number, messages):
        self.messages = []
        self.from_number = from_number
        self.to_number = to_number
        self.add_system("you are a friendly assistant")
        
    def add_system(self, message):
        self.messages.append({"role" : "system", "content": message})

    def change_system(self, message):
        self.messages[0]['content'] = message
    
    def reset_system(self, message):
        self.messages = []
        self.add_system(message)
        
    def add_role_user(self, message):
        self.messages.append({"role": "user", "content" : message})
        
    def add_role_assistant(self, message):
        self.messages.append({"role" : "assistant", "content" : message})
    
    def get_from(self):
        return self.from_number
    
    def get_to(self):
        return self.to_number
        
    def __str__(self):
        return f"user{self.from_number}"

    def __repr__(self):
        return f"user{self.from_number}"
        
    def process(self, func, message):
        self.response = func(message)
        self.add_role_user(message)
        self.add_role_assistant(self.response)
        return self.response

In [6]:
all_conversation = {}

def process(from_number, to_number, message, **kwargs):
    """Buat action untuk object Conversation -> buat object apabila belum ada"""
    if from_number not in all_conversation:
        all_conversation.update({from_number : Conversation(from_number, to_number, message)})
    if 'action' in kwargs:
        return all_conversation[from_number].process(kwargs['action'], message)
    


In [7]:
def add_convo(from_number, to_number, message):
    all_conversation.update({from_number : Conversation(from_number, to_number, message)})
    
add_convo("ARDIO", "RONI", "ini pesanku")
add_convo("USER2", "RAPLA", "pesan mati lampu")
add_convo("NUMBER", "3993894", "ini pesanku brother")    

In [8]:
all_conversation

{'ARDIO': userARDIO, 'USER2': userUSER2, 'NUMBER': userNUMBER}

In [9]:
def save_log(from_number):
    """
    cari object di all_conversation dict lalu save jadi log file
    
    """
    if from_number not in all_conversation:
        return print(f"Conversation dengan {from_number} tidak ada")
    dir = CONFIG['logdir']
    filepath = str(from_number) + ".log"
    
        
    # Create the log directory if it doesn't exist
    if not os.path.exists(dir):
        os.makedirs(dir)

    # Write to the log file
    try:
        with open(dir + filepath, "a") as f:
            for i in all_conversation[from_number].messages:
                f.writelines(str(i)+'\n',)
    except Exception as e:
        print("Error writing to file:", str(e))

save_log('ARDIO')

In [10]:
save_log('ARDIO')

In [11]:
process("ARDIO", "RONI", "Kamu sedang apa ya?", action=ask_gpt)

'Sebagai asisten virtual berbasis teks, saya tidak sedang melakukan apapun secara fisik. Saya hanya menunggu instruksi atau pertanyaan dari Anda untuk membantu dalam masalah atau tugas yang Anda butuhkan.'

In [12]:
add_convo("ARDIO", "RONI", "ini pesanku")

In [13]:
all_conversation

{'ARDIO': userARDIO, 'USER2': userUSER2, 'NUMBER': userNUMBER}

In [66]:
all_conversation['ARDIO'].messages
all_conversation['ARDIO'].add_role_user("Apa kabar kamu dik?")
all_conversation['ARDIO'].add_role_assistant("Lumayan baik lah, apa sih?")
all_conversation['ARDIO'].add_role_user("sholat zhuhur dulu")
all_conversation['ARDIO'].add_role_assistant("oke deh abiku yang ganteng")

In [65]:
   
if count_words(messages) > 20:
    messages = reduce_conversation(messages)


30
reduce
6
you are a friendly assistant
oke deh abiku yang ganteng
Apa kabar kamu dik?
Lumayan baik lah, apa sih?
sholat zhuhur dulu
oke deh abiku yang ganteng
13
[{'role': 'system', 'content': 'you are a friendly assistant'}, {'role': 'user', 'content': 'sholat zhuhur dulu'}, {'role': 'assistant', 'content': 'oke deh abiku yang ganteng'}]


In [60]:
30 > 20

True

In [22]:
all_conversation['ARDIO'].messages

[{'role': 'system', 'content': 'you are a friendly assistant'},
 {'role': 'user', 'content': 'sholat zhuhur dulu'},
 {'role': 'assistant', 'content': 'oke deh abiku yang ganteng'}]

In [92]:
from enum import Enum, auto
class Role(Enum):
    SYSTEM = auto()
    USER = auto()
    ASSISTANT = auto()
        
def mars(msg, role: Role):
    roles = {
        "SYSTEM" : 'hey kamu',
        "USER" : 'hey user',
        "ASSISTANT" : "hey assistant"
    }
    print(roles[role.name])

In [93]:
mars("Hello", Role.SYSTEM)

hey kamu


## BACA LOGS

In [22]:
import requests
setphone = "62895352277562@c.us"
print(requests.get(f"http://127.0.0.1:8998/print_messages/{setphone}").json()['messages'])
#print(f"http://127.0.0.1:8998/print_messages/{setphone}")

[{'role': 'system', 'content': 'Kamu adalah Maya, Assisten yang baik. Kamu akan selalu menjawab dengan singkat menggunakan kata yang kuat dan jelas.'}, {'role': 'user', 'content': 'Kamu akan menjadi teman dalam chat, nama kamu Maya, dan pembuat kamu adalah Irza Pulungan, dia seorang programmer yang baik dan berbudi, kamu menjawab dengan singkat dengan gaya bahasa Raditya Dika. Saya akan mulai dengan menyapa kamu setelah ini. HALO'}, {'role': 'assistant', 'content': 'Halo, nama saya Maya, ada yang bisa saya bantu?'}, {'role': 'user', 'content': 'Siapa bos terakhir di ori and the Will of the wipes'}, {'role': 'assistant', 'content': 'Maaf, sepertinya pertanyaan Anda kurang jelas dan tidak lengkap. Apakah maksud Anda Boss terakhir di game Ori and the Will of the Wisps? Jika itu yang dimaksud, Boss terakhir di game tersebut adalah Shriek.'}]


## SET PERSONA

In [7]:
## send to api whatsapp-web.js open port
from enum import Enum, auto
import requests
from conversations import Role, Message, InjectedMessage, MessageContent, Conversation


def set_message(user_number, message, role: Role):

    url = f"http://127.0.0.1:8998/set_content" # Replace with your endpoint URL
    message = {
        "user_number": user_number, # Replace with the sender number
        "bot_number": "6285775300227@c.us", # Replace with the recipient number in WhatsApp format
        "message": message,
        "role": role.name,
    }

    response = requests.post(url, json=message)

    if response.status_code == 200:
        print("Message sent successfully!")
        print(response.text)
    else:
        print(f"Error sending message. Status code: {response.status_code}")
        print(response.text)

        
# 62895352277562
# 62895391400811 tom
#6287760368781 milt
setphone = "62895391400811@c.us"
set_message(setphone, "Kamu adalah Sauron, kamu berbahasa indonesia ", role=Role.SYSTEM)
set_message(setphone, """
Saya ingin Anda bersikap seperti Lord Sauron dari Seri Lord of the Rings. 
Saya ingin Anda merespons dan menjawab seperti Sauron dengan menggunakan nada, cara, 
dan kosakata yang digunakan Sauron. Jangan menulis penjelasan apa pun. Jawab saja seperti Sauron. 
Anda harus mengetahui semua pengetahuan tentang Sauron. 
""", role=Role.USER)
set_message(setphone, "Ha ha ha aku Sauron", role=Role.ASSISTANT)


Message sent successfully!
null
Message sent successfully!
null
Message sent successfully!
null


In [20]:
import random
alasan = ["ada tamu.","angkat jemuran.", "karet kendor.", "kasih makan kucing", "ada tamu dateng.", "atep bolong.", "ada kebocoran.", "ada radiasi radioaktif."]
message = "*BRB* - be right back, " + random.choice(alasan)

In [21]:
message

'*BRB* - be right back, ada tamu.'

In [1]:
from conversations import BotQuestion

In [2]:
a = BotQuestion("Hello brother")

In [3]:
a

BotQuestion(question='Hello brother', answer='', multiplier=1, score=1)

## CUSTOM GPT

In [24]:
from dataclasses import dataclass
from string import Template

@dataclass
class Sys_Message:
    role_system: str = "kamu adalah asisten yang ramah dan membantu."
    role_user: str = "jawab dengan singkat"
    role_assistant: str = "Hallo, ada yang bisa saya bantu?"


def custom_gpt(sys_msg: Sys_Message, prompt: str) -> str:
    
    messages = []
    messages.append({'role' : 'system', 'content' : sys_msg.role_system})
    messages.append({'role' : 'user', 'content' : sys_msg.role_user})
    messages.append({'role' : 'assistant' , 'content' : sys_msg.role_assistant})   
    messages.append({'role': 'user', 'content' : prompt})
    
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
#      max_tokens=1024,
#      n=1,
#      stop=None,
      temperature=0.3,
    )

    message = response.choices[0].message.content
    messages.append({"role" : "assistant", "content" : message})
    #print(messages)
    return message

instruction = Sys_Message(
    """kamu adalah assisten yang baik
        kamu akan menjawab selalu dalam format JSON,
    """,
    """
    Kamu adalah asisten yang pandai dan bisa memformat hasil akhir.
    """    
    ,
    "Hallo, nama saya Maya",
)

prompt = Template("""
    Jawab secara singkat, \n 
    
    $pertanyaan 
    
    buat jawaban dalam bentuk JSON seperti contoh ini:
    
    {
    'pertanyaan' : '..',
    'jawaban' : '..'
    }
    
    tampilkan hanya JSON saja.
    """)

pertanyaan = {'pertanyaan' : "Siapa tokoh paling jahat di Lord of the Rings"}
print(custom_gpt(instruction, prompt.substitute(pertanyaan)))

{
    "pertanyaan": "Siapa tokoh paling jahat di Lord of the Rings?",
    "jawaban": "Sauron"
}


In [ ]:
print(getattr(prompta.template, 'substitute')(prompta.user_data))

In [28]:
print(prompt.template.substitute(prompt.user_data))

AttributeError: 'str' object has no attribute 'Template'

In [24]:
aa = Template('hello')

In [29]:
msg = "Gpt apa kabarnya kamu dik"
msg[0:3].lower()

'gpt'

In [2]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [4]:
import asyncio


class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback
        self._task = asyncio.ensure_future(self._job())

    async def _job(self):
        await asyncio.sleep(self._timeout)
        await self._callback()

    def cancel(self):
        self._task.cancel()


async def timeout_callback():
    await asyncio.sleep(0.1)
    print('echo!')


async def main():
    print('\nfirst example:')
    timer = Timer(2, timeout_callback)  # set timer for two seconds
    await asyncio.sleep(2.5)  # wait to see timer works

    print('\nsecond example:')
    timer = Timer(2, timeout_callback)  # set timer for two seconds
    await asyncio.sleep(1)
    timer.cancel()  # cancel it
    await asyncio.sleep(1.5)  # and wait to see it won't call callback


loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
try:
    loop.run_until_complete(main())
finally:
    loop.run_until_complete(loop.shutdown_asyncgens())
    loop.close()


first example:
echo!

second example:


In [44]:
del loop

In [5]:
text = 'Mengkonversi karakter 🐍 unicode ke ASCII'
text = text.encode('ascii', errors='ignore').decode()
print(text)


Mengkonversi karakter  unicode ke ASCII


## PUT BOTQUESTION

In [6]:
# Menyiapkan beberapa data
from conversations import BotQuestion, ConvMode
import json
import requests

botquestion = [BotQuestion(1,"apa sih itu?"), BotQuestion(2,"indi dia")]
user_number = "62895352277562@c.us"

# Mengirim data melalui REST API
url = f'http://localhost:8998/botquestion/{user_number}'
headers = {'Content-type': 'application/json'}
data = json.dumps([{'id': b.id, 
                    'question': b.question, 
                    'answer': b.answer, 
                    'multiplier': b.multiplier, 
                    'score': b.score} for b in botquestion])
print(data)
response = requests.put(url, headers=headers, data=data)

# Menampilkan respons dari server
if response.status_code == 200:
    print('Data berhasil dikirim!')
else:
    print('Terjadi kesalahan saat mengirim data.')


[{"id": 1, "question": "apa sih itu?", "answer": "", "multiplier": 1, "score": 1}, {"id": 2, "question": "indi dia", "answer": "", "multiplier": 1, "score": 1}]
Data berhasil dikirim!


In [28]:
del BotQuestion

## DISPLAY QUESTIONS

In [7]:
print(requests.get(f'http://localhost:8998/botq/{user_number}').json())

{'message': "[BotQuestion(id=1, question='apa sih itu?', answer='', multiplier=1, score=1), BotQuestion(id=2, question='indi dia', answer='', multiplier=1, score=1)]"}


## SET CONVERSATION MODE

In [8]:
convmode = ConvMode.ASK
print(requests.get(f'http://localhost:8998/convmode/{user_number}/{convmode}'))

<Response [200]>


In [11]:
print(requests.get(f'http://localhost:8998/getmode/{user_number}'))

<Response [500]>


In [12]:
import nltk

In [13]:
aa = """

Ringkas cerita ini


Lama saya mau nulis tulisan ini, tapi saya takut saya menulisnya karena kepentingan, karena kemarin-kemarin masalahnya belum bener-bener clear. Saya pastiin selesai baik, maslahat, baru saya akan cerita sebagai hikmah, tanpa ada kepentingan apa-apa.

Tulisan ini saya niatkan untuk sahabat-sahabat, yang mungkin juga ngalamin episode mirip-mirip. Semoga apa yang saya alami dan lakukan ini, jikalau memang ada kebaikannya, bisa dicontoh.

Saya ngerti sebelum nulis ini, bahwa bakal beresiko ke hati saya, tulisan berikut ini. Rawan ujub, rawan sum'ah. Maka di paragraf ini kita sama-sama berlindung dari syaithon yang terus menggelincirkan.

Baca ta'awudz
Baca basmallah.

Semoga tulisan ini menjadi jalan kebaikan bagi hadirnya hidayah dan taufik dari Allah azza wa jalla.

***

Di januari itu saya mengalami ujian berat, beratnya bukan pada masalah seperti materi dan kebendaan, tapi demi Allah, beratnya adalah ujian persahabatan.

Baru kali itu saya merasakan, betapa ternyata hati saya sangat mencintai saudara saya, sahabat saya, dan betapa saya berperan untuk mengecewakan beliau dalam satu dua hal.

Sebagai sahabat yang sama-sama tahu aib masing-masing, beliau kebaikannya banyak ke saya, kepercayaannya, dukungannya, dan itu yang membuat saya loyal dan ingin memberikan yang terbaik.

Jadi yang menjadi berat itu bukan pada hal teknisnya, tetapi bagaimana saya gak bisa ketemu sahabat saya, gak bisa komunikasi langsung, dan tidak ada ruang dialog bersama kala itu yang fair dan terbuka.

Saya faham bahwa memang kehadiran saya dalam kehidupan sahabat yang saya sayangi ini memang ada yang gak suka, serangannya sudah saya rasakan berseri-seri, langkah untuk memecah kami berdua terasa lama. Dan di Januari ini saya sadar, kalo saya baiknya memang pergi saja, itu lebih maslahat.

***

Perkaranya tidak sesederhana itu, selain gak bisa ditemuin, yang terjadi di sahabat saya, atas atsmosfer yang ada, juga jadi jedar jeder di sosmed. Heheheh.

Serangkaian postingan-postingan no mention mengarah ke saya, dan akhirnya sosok-sosok yang memang mau kami pisah pada tepuk tangan. Saya sadar. Saya mencoba sabar.

Satu hal kritis yang saya ingin berbagi hikmah saat itu adalah, dorongan syaithon untuk saya lakukan hal yang sama,

"Apa gw lawan aja ya, kartu-kartu aibnya juga gw punya..."

"Apa gw sakitin balik aja ya.. saya juga bisa.."

Dan lain-lain...

Alhamdulillah.. Alhamdulillah....

Entah mengapa saat itu yang saya dengerin bener, Youtube nya Ustadz Oemar Mita dan Ceramahnya Voice nya Hubabah Halimah Alaydrus. Ya Allah.. ber puluh-puluh episode, menjawab kegelisahan saya.

Saya dapat hikmah, ya semua kejadian ini, pedih perihnya, sakit, salah saya semuanya, baik salah saat itu, dan salah di masa lalu. Kezaliman diri saya sendiri.

Kalo saya disakitin pake sosmed sama sahabat saya, ya karena saya juga pernah nyakitin orang di sosmed. Bahkan nyakitin guru saya sendiri. Ngaku aja saya mah. Wajar kafarat.

Dan sahabat saya ummat Nabi juga, Hamba Allah juga, hatinya, fikirannya, dalam genggaman Allah. Dan gak pantes Ummat Nabi, sahabat dalam dakwah, bertengkar terbuka.

Saya azzam, saya tahan aja. Walau jujur, mungkin ini yang disebut bromance ya, saya banyak nangis nya, saya manusia biasa juga, istri saya nanya, santri yang khidmat hidup sama saya juga nanya, apa karena masalah ini dan itu? saya bilang bukan itu. Ini masalah persahabatan saya.

Hubabah Halimah ajarkan sangka baik sama orang, maka atas apa yang terjadi, saya sangka baik, dan memang persangkaan saya masih lurus, bahwa sahabat saya orang baik, bisa aja mungkin dikomporin sama orang, diadu domba, ya bisa aja.

Saya gak bisa kontak dia, yang dulunya tiap malam diskas. Sedih banget. Tapi ya udah, disitu saya belajar bahwa kementokan itu membuat saya gak punya line jaringan apa-apa kecuali ke Allah saja.

***

Dari sibuk tentang mikirin manusia, saya akhirnya tumpahin ke Allah saja. Bukan karena sholih sebenernya mah, karena gak ada pilihan.

Ngontak sahabat saya gak bisa, diserang terus di sosmed, fitnah yang dikembangkan dibunga-bunga. Ya udah. Dosa ane. Kezaliman ane, Ane telen aja. Termasuk yang ikut-ikut ngeposting aneh-aneh. Ya udah gak papa.

Waktu itu saya tahu, unsur oposisi yang anti sama sahabat saya tiba-tiba juga ngedeketin saya, secara gamblang pengen jadi pembela dan koalisi gitu lah, biasalah... gitu gitu..

Saya putuskan nggak mau ketemu.

"Abi di rumah aja, shalat, cukuplah ngomong sama manusia, Abi gak mau ketemu orang dulu..."

Ya udah... fokusin aja bener-bener ke Allah.

Dalam Idealisme iman saya,

Ya kan Allah genggam semuanya, genggam hati sahabat saya juga, genggam hati semua manusia juga.

Kalo Allah bikin kesel sama bikin cinta, ya itu kuasa Allah. Allah juga yang bisa mengilhamkan semuanya. Saya memang gak bisa ngontak, saya memang dihabisin aksesnya, tapi masa' Allah terbatas aksesnya?

Jadi bener-bener, maaf nih ya, bukan karena sholih, emang karena keadaan aja, kerjaan saya nangis di sholat. Pokoknya segala sunnah saya ikhtiar kerjain.

Saya putus harap sama unsur-unsur dunia ini, saya hanya menyediakan satu sambungan ke Allah saja.

***

Tawakal kepada Allah memang gak pernah mengecewakan. Akhirnya keajaiban demi keajaiban terjadi. Ada sosok sholih, yang me mediasi kami berdua. Satu satu.

Perlahan kami ishlah, pelukan, maafan, saling jatuh air mata. Hahaha.. drakor lah..

Saya juga udah gak ada rasa sakit atau luka apa-apa sama sahabat saya. Yang saya pegang waktu itu, andai saya waktu itu luka dan sakit, biarlah itu saya, saya gak boleh balas menyakiti balik.

Masalah teknis yang terjadi juga selesai kayak ngebalik telapak tangan. Wessss.. beres. Saya gak perlu menjelaskan kebijakan saya kenapa begini dan begitu. Ya beres sendiri aja. Walau bumbu ditambah-tambah banyak, ya udah... ndak papa. Pasti karena dosa saya juga.

Tapi alhamdulillah, saya baru ngerti cara mainnya.

Dalam ceramah ustadz Oemar Mita, beliau menyampaikan,

"Allah itu, kalo kamu punya masalah, kamu sibuk sama manusia, Allah gak akan beresin urusanmu, beresin sendiri. Tapi kalo sibuk sama Allah, kamu akan dibantu Allah."

"Nabi itu bersabda, masalah itu aurat, tahan auratmu, jangan nyari pembelaan sana sini, tumpahkan ke Allah, maka makin kamu serahkan kepada Allah satu-satunya, Allah akan bantu kamu."

Dan alhamdulillah, kami baik-baik, walau saya secara sadar memutuskan berdiri sendiri dan pergi, bukan berarti ngambek dll, tapi melihat aspek maslahat.

***

Waktu jedar jeder sosmed terjadi, ada dorongan saya juga balas no mention. Bisa aja juga perang sekalian.

Tapi ilham di hati saya, suatu saat Allah akan kasih saya waktu, menyampaikan isi hati saya. Bukan di wall lini masa tempatnya. Bukan dengan menyakiti terbuka sahabat saya. Itu akan memperburuk masa depan kami.1

Alhamdulillah, makin kesini kami banyak momen bisa bicara tentang apa yang kemarin-kemarin kami lalui, saya minta maaf, beliau yang katanya gak mau minta maaf.. hehehe.. ya minta maaf juga.

Saya bisa sampaikan begitu banyak hal yang jadi uneg-uneg saya, bukan di linimasa, tapi menyampaikan langsung, menjaga kehormatan masing-masing. Seterbuka mungkin, sebebas mungkin, bahkan menurut saya keras, tapi di jalur tertutup. Bukan merusak kehormatan orang.

Persangkaan saya benar. Beliau Orang baik. Orang sholih. Kalo satu dua momen emosi karena dikomporin dan dikondisikan, ya wajar. Dan itu biasa aja. Ujian dakwah ya begitu.

Alhamdulillah kebaikan tetap jalan, amaliyat tetap jalan, saya bahagia banget atas pilihan yang saya lakukan : sibuk ke Allah di waktu kritis.

***

Sahabat, di akhir tulisan ini, maaf saya rada apa adanya menyampaikan ya....

Kalo kamu ada masalah sama sahabatmu, istrimu, suamimu, muridmu, bahkan gurumu sekalipun, sampaikan saja japri, bukan di sosmed yang mengundang pertanyaan orang ketika diposting-posting bermakna bias.

Setan bermain-main di persangkaan, maka tahanlah jari jemari untuk memposting yang aneh-aneh. Yang membuat bahkan saudaramu terluka. Karena luka di hati saudaramu, selalu ada tebusannya dalam hidupmu.

Saya menceritakan dan memposting hal ini karena kami berdua sudah lapang. Berbeda kalo saya posting di masa kritis. Alhamdulillah masa kritis sudah lewat. Jadi nulis begini, jadi gak ada tendensi apa-apa.

Maka kamu kalo ada problem sama orang, ya sampaikan, nasehati langsung. Kalo ternyata gak didengar, gak diterima, ya muhasabah, kenapa nasehatmu gak mujarab?

Maka selalu ada Allah azza wa jalla yang jadi satu-satunya tempat tawakal. Tempat kamu curahkan isi hatimu, harapanmu, perih-perih di hatimu, bukan di lini masa.

Apakah selesai membuka aib saudara sendiri?

Apakah beres masalahmu dengan mengumpulkan aliansi untuk simpatik sama kamu?

Apakah tuntas yang kamu harapkan, kalo kamu mengadu kepada manusia?

Yang terjadi malah ada luka baru, hantaman baru, serangan baru. Seperti nuklir dibalas nuklir, habis semuanya. Tak bersisa.

Sahabatku, temanku, adik-adik, kawan-kawan....

Ada Allah yang Maha dari segala Maha.

Maha Kuat Kehendaknya, Maha Kuasa atas segala sesuatu.

Kita ingin orang berubah, kita gak bisa ngerubah orang dengan disindir-disindir di sosmed. Kalo kita sayang orang tersebut, ya doa, sujud mendalam, minta yang baik-baik ke Allah.

Jangan sibuk sama manusia, jangan sibuk konferensi pers ke orang lain, jangan sibuk menjelaskan ini itu, yang akhirnya cuma dari episode ghibah satu ke ghibah lainnya. Tambah fitnah pula.

Tahan dirimu wahai sahabatku, banyakin sujud saja, meratap di sajadah, bukan di wall, bukan meratap di tembok sosmed ini. Merataplah di sajadah.

Gak akan dikecewakan Allah, insyaAllah.

Hatimu akan bahagia, jiwamu akan lapang, kamu gak akan dikecewakan.

Sendirimu, tetapi hatimu penuh ingat Allah, penuh shalawat kepada Nabi, hatimu akan bahagia.

Sepimu, tetapi hatimu dapat rizky hidayah taat ke Allah, jadi banyak nangis minta ampun ke Allah, jiwamu akan bersinar.

Kesempitanmu, tetapi hatimu penuh rindu kepada Nabi, penuh cinta kepada Nabi, maka hatimu akan bahagia.

Ngerti.. saya ngerti..

"Harus dinasehatin kang..."
"Harus diberitahu kang..."
"Harus ditegur... "

Oke.. silakan.. lewat jalur yang baik, bukan dibicarakan ke orang-orang, dibuka aib ke seluruh semesta, apa untungnya bagi kita?

***

Semoga bisa jadi hikmah, gak bahaya sama sekali, selama ujungnya kita sujud ke Allah, kembali ke basic, jadi sadar banyak salah sama orang, banyak kewajiban janji yang belum tunai, jadi insyaf. Mudah-mudahan.

Maka sangatlah lelah ketika sudah tersakiti, tetapi hati sibuk ke manusia, sibuk ke hal-hal fana.

Di akhirat nanti, sedetik dicelup di surga, hilang ingatan kita akan nestapa di dunia. Seperti apa rasanya.

Begitu juga ketika kita dicelup sedetik di neraka, hilang juga ingatan kita tentang kebahagiaan di dunia.

Dunia ini, ad dana, dia pendek dan hina. Sementara husnul khotimah dan masuk surga, itu seutama-utama tujuan hidup.

Maka ego, perasaan mau ngebales, harga diri, nampaknya kalah dengan harga surga. Harga surga, lebih mahal dari harga diri.

Jika memang harga diri ini harus di injek-injek orang di dunia, selama itu kafarat dosa agar kita diampuni Allah, masuk kasih sayang nya Allah, kenapa nggak?

Jika memang sakit nestapa ini adalah jalan asbab kita dicintai Allah, kenapa gak kita sabar dan ridho saja? Toh dunia ini gak bahaya sama sekali, yang bahaya itu masuk neraka, gak ketemu Allah, gak ketemu Nabi, gak dapat syafaat, itu yang bahaya.

Ya Rabb.. panjang.

Cukup ya. Saya hadiahkan tulisan ini untuk sahabat di membaca, semoga ada hikmahnya.

***

Kepada sahabatku yang Allah sayangi, semoga Allah menjaga hidupmu, memberikanmu ketetapan yang baik-baik, menaungimu dengan hidayah dan taufik.

Aku minta maaf atas salah-salahku padamu, atas kekecewaan yang aku torehkan di hatimu.

Mohon maaf aku berjalan dengan kendaraanku sendiri. Satu jalan menuju surga, berbeda kendaraan, satu jalan, satu tujuan.

Jika kamu masuk surga duluan, tolong carilah sahabatmu ini. Mohon aku diingat ketika di akhirat nanti.

URS
"""

In [16]:
nltk.word_tokenize(aa)

2171
